In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def create_mock_analytics_data(n_customers=1000, n_products=200, n_purchases=5000):
    np.random.seed(42)
    random.seed(42)
    
    # Customer DataFrame
    customers = pd.DataFrame({
        'customer_id': range(1, n_customers + 1),
        'age': np.random.randint(18, 80, n_customers),
        'gender': np.random.choice(['M', 'F'], n_customers),
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], n_customers),
        'state': np.random.choice(['CA', 'TX', 'NY', 'FL', 'IL'], n_customers),
        'income': np.random.normal(65000, 25000, n_customers).astype(int),
        'signup_date': pd.date_range('2020-01-01', '2024-01-01', periods=n_customers),
        'customer_segment': np.random.choice(['Premium', 'Standard', 'Basic'], n_customers),
        'preferred_channel': np.random.choice(['Online', 'Store', 'Mobile'], n_customers),
        'loyalty_score': np.random.randint(1, 101, n_customers)
    })
    
    # Product DataFrame
    products = pd.DataFrame({
        'product_id': range(1, n_products + 1),
        'category': np.random.choice(['Electronics', 'Clothing', 'Home', 'Sports'], n_products),
        'brand': np.random.choice(['Apple', 'Samsung', 'Nike', 'Adidas', 'IKEA'], n_products),
        'price': np.round(np.random.exponential(50, n_products) + 10, 2),
        'cost': np.round(np.random.exponential(30, n_products) + 5, 2),
        'rating': np.round(np.random.uniform(1, 5, n_products), 2),
        'stock_quantity': np.random.randint(0, 1000, n_products)
    })
    
    # Purchase DataFrame
    purchases = pd.DataFrame({
        'purchase_id': range(1, n_purchases + 1),
        'customer_id': np.random.choice(customers['customer_id'], n_purchases),
        'product_id': np.random.choice(products['product_id'], n_purchases),
        'purchase_date': pd.date_range('2020-01-01', '2024-12-01', periods=n_purchases),
        'quantity': np.random.poisson(2, n_purchases) + 1,
        'channel': np.random.choice(['Online', 'Store', 'Mobile'], n_purchases),
        'discount_applied': np.round(np.random.uniform(0, 0.3, n_purchases), 2),
        'returned': np.random.choice([True, False], n_purchases, p=[0.08, 0.92])
    })
    
    # Add total amount
    purchases = purchases.merge(products[['product_id', 'price']], on='product_id')
    purchases['total_amount'] = np.round(purchases['quantity'] * purchases['price'] * (1 - purchases['discount_applied']), 2)
    
    return customers, products, purchases

In [3]:
customers_df, products_df, purchases_df = create_mock_analytics_data()

In [103]:
customers_df

,customer_id,age,gender,city,state,income,signup_date,customer_segment,preferred_channel,loyalty_score
0,1,56,M,Phoenix,IL,43243,2020-01-01 00:00:00.000000000,Basic,Store,98
1,2,69,M,New York,FL,59657,2020-01-02 11:05:56.756756756,Basic,Mobile,77
2,3,46,F,Chicago,IL,79007,2020-01-03 22:11:53.513513513,Basic,Store,58
3,4,32,M,Los Angeles,TX,49609,2020-01-05 09:17:50.270270270,Basic,Online,78
4,5,60,M,Chicago,FL,82346,2020-01-06 20:23:47.027027027,Basic,Store,75
...,...,...,...,...,...,...,...,...,...,...
995,996,18,M,New York,TX,103389,2023-12-26 03:36:12.972972960,Standard,Store,44
996,997,35,F,Phoenix,TX,78468,2023-12-27 14:42:09.729729728,Premium,Store,64
997,998,49,F,Chicago,NY,59718,2023-12-29 01:48:06.486486480,Premium,Mobile,31
998,999,64,M,Chicago,CA,54000,2023-12-30 12:54:03.243243232,Premium,Online,47


In [104]:
products_df

,product_id,category,brand,price,cost,rating,stock_quantity
0,1,Sports,Apple,28.20,84.09,4.30,269
1,2,Clothing,Nike,19.25,7.04,3.59,291
2,3,Electronics,Nike,40.87,28.92,4.88,452
3,4,Clothing,Samsung,28.34,31.81,3.86,714
4,5,Electronics,Apple,11.79,36.73,4.70,23
...,...,...,...,...,...,...,...
195,196,Sports,Adidas,209.29,28.17,2.33,707
196,197,Home,Samsung,15.12,7.19,3.98,445
197,198,Clothing,Samsung,28.05,86.56,1.61,207
198,199,Home,IKEA,60.84,48.62,1.52,469


In [105]:
purchases_df

,purchase_id,customer_id,product_id,purchase_date,quantity,channel,discount_applied,returned,price,total_amount
0,1,976,82,2020-01-01 00:00:00.000000000,4,Mobile,0.14,False,12.58,43.28
1,2,819,131,2020-01-01 08:37:21.088217643,3,Store,0.27,False,35.75,78.29
2,3,104,118,2020-01-01 17:14:42.176435287,5,Online,0.17,False,17.29,71.75
3,4,742,81,2020-01-02 01:52:03.264652930,2,Mobile,0.04,False,48.85,93.79
4,5,426,70,2020-01-02 10:29:24.352870574,4,Store,0.16,False,41.40,139.10
...,...,...,...,...,...,...,...,...,...,...
4995,4996,91,182,2024-11-29 13:30:35.647129408,1,Store,0.13,False,30.59,26.61
4996,4997,854,16,2024-11-29 22:07:56.735347072,3,Online,0.11,False,81.26,216.96
4997,4998,272,183,2024-11-30 06:45:17.823564704,3,Store,0.07,False,10.91,30.44
4998,4999,834,94,2024-11-30 15:22:38.911782336,2,Mobile,0.10,True,24.70,44.46


In [106]:
# Usage
# from broinsight.metadata.metadata_generator import DataFrameMetadataGenerator
# generator = DataFrameMetadataGenerator()
# generator.generate_metadata(customers_df, "customers_df", "Customer information with loyalty score")
# generator.generate_metadata(products_df, "products_df", "Product catalog with pricing and inventory")
# generator.generate_metadata(purchases_df, "purchases_df", "Customer purchases and return status")


In [4]:
from broinsight.metadata.metadata_loader import MetadataLoader

# Usage
loader = MetadataLoader("metadata")

# Load all tables
full_context = loader.construct_prompt_context()

# Load specific tables only
selected_context = loader.construct_prompt_context(["customers_df", "products_df"])


In [108]:
print(loader.get_summary_prompt())

Table: purchases_df - Description: **Table: Purchases and Returns**

This table records every customer order and whether the order was returned. It includes the date and time of the sale, the product bought, the quantity, the price, any discount applied, the total amount paid, the channel through which the purchase was made (online, store, mobile), and the customer who made the purchase.  

**Primary purpose / use cases**  
- **Sales performance** – view total revenue, average sale size, and top‑selling products.  
- **Channel analysis** – compare how many sales occur in each channel and how channel mix affects revenue.  
- **Return monitoring** – calculate return rates, identify high‑return products, and assess the impact of returns on cash flow.  
- **Discount effectiveness** – see how discounts influence the quantity sold and the final revenue.  
- **Revenue forecasting** – feed into budgeting and planning by linking past sales, discounts, and returns to future projections.  

**Key

In [109]:
print(full_context)

TABLE: purchases_df
DESCRIPTION: **Table: Purchases and Returns**

This table records every customer order and whether the order was returned. It includes the date and time of the sale, the product bought, the quantity, the price, any discount applied, the total amount paid, the channel through which the purchase was made (online, store, mobile), and the customer who made the purchase.  

**Primary purpose / use cases**  
- **Sales performance** – view total revenue, average sale size, and top‑selling products.  
- **Channel analysis** – compare how many sales occur in each channel and how channel mix affects revenue.  
- **Return monitoring** – calculate return rates, identify high‑return products, and assess the impact of returns on cash flow.  
- **Discount effectiveness** – see how discounts influence the quantity sold and the final revenue.  
- **Revenue forecasting** – feed into budgeting and planning by linking past sales, discounts, and returns to future projections.  

**Key r

In [110]:
print(selected_context)

TABLE: customers_df
DESCRIPTION: 
This table holds the key details about each customer, including who they are, where they live, how much they earn, and how loyal they are to the brand. It is used by marketing, sales, and product teams to segment customers, target promotions, and measure loyalty. The data links directly to customer‑service interactions, purchase history, and campaign results, helping the company tailor offers, forecast revenue, and improve retention. Typical users are marketing analysts, loyalty program managers, and senior executives who need a clear view of customer profiles and engagement levels.
FIELDS:
  - age (INTEGER): Customer age in years (18-79)
  - city (VARCHAR): Customer residential city [Values: Chicago, Houston, Los Angeles, New York, Phoenix]
  - customer_id (INTEGER): Unique customer identifier
  - customer_segment (VARCHAR): Customer tier classification (Basic, Standard, Premium) [Values: Basic, Premium, Standard]
  - gender (VARCHAR): Customer gender

In [111]:
from broinsight.metadata.table_descriptor import TableDescriptor
from broprompt import Prompt

In [112]:
td = TableDescriptor(
    Prompt.from_markdown("./broinsight/prompt_hub/table_descriptor.md").str, 
    OpenAI20b()
)

In [113]:
metadata = loader.get_metadata_dict()['customers_df']
print(td.run("{metadata}".format(metadata=metadata)))


The customers_df table stores the core profile information for every customer in the business.  
It includes who the customer is (ID, gender, age), where they live (city, state), how much they earn (annual income), how loyal they are (loyalty score), what tier they belong to (Basic, Standard, Premium), what channel they prefer for shopping, and when they first signed up.  

This data is used by marketing, sales, and product teams to:
- Segment the customer base for targeted campaigns and promotions
- Personalize offers and communications based on demographics, income, and loyalty
- Forecast revenue and assess the impact of marketing activities
- Measure and improve customer retention and satisfaction  

The table connects to other business processes such as customer‑service interactions, purchase history, and campaign analytics, allowing a holistic view of each customer’s journey.  

Typical users are marketing analysts, loyalty program managers, product managers, and senior executive

In [114]:
metadata_dict = loader.get_metadata_dict()
for table_name, detail in metadata_dict.items():
    description = td.run("{metadata}".format(metadata=detail))
    metadata_dict[table_name]['description'] = description

In [115]:
import yaml

for table_name, detail in metadata_dict.items():
    with open(f"metadata/{table_name}.yaml", "w") as f:
        detail.update({"table_name": table_name})
        yaml.dump(detail, f)

In [10]:
from brollm import BedrockChat, BaseLLM

class BaseOpenAI(BedrockChat):
    def __init__(self, model_name):
        super().__init__(model_name=model_name)
    
    def run(self, system_prompt, messages):
        model = self.get_model()
        response = model.converse(
            modelId=self.model_name,
            messages=messages,
            system=self.SystemMessage(text=system_prompt),
            inferenceConfig={
                # "maxTokens": 150, 
                "temperature": 0.7, 
                # "topP": 0.9
            },
        )
        return response['output']['message']['content'][-1]['text']

class OpenAI20b(BaseOpenAI):
    def __init__(self):
        super().__init__(model_name="openai.gpt-oss-20b-1:0")

In [117]:
from broinsight.flow import get_flow
from broinsight.actions import Shared
from broflow import state

state.set("debug", False)

flow = get_flow(model=OpenAI20b())
flow.save_mermaid("flow.md")

import duckdb
from broinsight.metadata.metadata_loader import MetadataLoader
loader = MetadataLoader("metadata")

conn = duckdb.connect()
conn.register("customers_df", customers_df)
conn.register("products_df", products_df)
conn.register("purchases_df", purchases_df)
shared = Shared(db=conn, metadata_loader=loader)

flow.run(shared)

Welcome to BroInsight!
AI: Sure thing! If you’d like to end the session, just close the chat window or tab. I’m here if you have any more questions later on. Have a great day!


KeyboardInterrupt: 

In [ ]:
print(shared.sql_query)
conn.execute(shared.sql_query).df()


SELECT p.*
FROM products_df p
JOIN (
    SELECT category, MAX(price) AS max_price
    FROM products_df
    GROUP BY category
) pm
  ON p.category = pm.category
 AND p.price   = pm.max_price
ORDER BY p.category;



,product_id,category,brand,price,cost,rating,stock_quantity
0,157,Clothing,Apple,223.20,48.68,3.77,552
1,53,Electronics,Apple,386.26,18.36,1.75,633
2,193,Home,Samsung,193.97,14.36,3.99,559
3,47,Sports,IKEA,236.07,30.82,3.91,415


In [ ]:
products_df.query("product_id==157")

,product_id,category,brand,price,cost,rating,stock_quantity
156,157,Clothing,Apple,223.2,48.68,3.77,552


In [ ]:
purchases_df

,purchase_id,customer_id,product_id,purchase_date,quantity,channel,discount_applied,returned,price,total_amount
0,1,976,82,2020-01-01 00:00:00.000000000,4,Mobile,0.14,False,12.58,43.28
1,2,819,131,2020-01-01 08:37:21.088217643,3,Store,0.27,False,35.75,78.29
2,3,104,118,2020-01-01 17:14:42.176435287,5,Online,0.17,False,17.29,71.75
3,4,742,81,2020-01-02 01:52:03.264652930,2,Mobile,0.04,False,48.85,93.79
4,5,426,70,2020-01-02 10:29:24.352870574,4,Store,0.16,False,41.40,139.10
...,...,...,...,...,...,...,...,...,...,...
4995,4996,91,182,2024-11-29 13:30:35.647129408,1,Store,0.13,False,30.59,26.61
4996,4997,854,16,2024-11-29 22:07:56.735347072,3,Online,0.11,False,81.26,216.96
4997,4998,272,183,2024-11-30 06:45:17.823564704,3,Store,0.07,False,10.91,30.44
4998,4999,834,94,2024-11-30 15:22:38.911782336,2,Mobile,0.10,True,24.70,44.46


In [ ]:
# Find all customers with max purchases per year (including ties)
yearly_counts = (purchases_df.assign(year=purchases_df['purchase_date'].dt.year)
                 .groupby(['year', 'customer_id']).size()
                 .reset_index(name='purchase_count'))

top_buyers_with_ties = (yearly_counts
                       .groupby('year')
                       .apply(lambda x: x[x['purchase_count'] == x['purchase_count'].max()])
                       .reset_index(drop=True))

# print(top_buyers_with_ties)
top_buyers_with_ties


/var/folders/_p/hnd9c4p5193b63rmd117strw0000gn/T/ipykernel_98116/2640165579.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[x['purchase_count'] == x['purchase_count'].max()])


,year,customer_id,purchase_count
0,2020,257,5
1,2020,317,5
2,2020,783,5
3,2020,982,5
4,2021,417,7
5,2022,836,6
6,2022,899,6
7,2023,348,5
8,2023,757,5
9,2024,360,5


In [ ]:
t = []

In [ ]:
t.copy()

[]

In [12]:
from broinsight.actions import Shared

shared = Shared(method="ask")

In [ ]:
from broinsight import BroInsight
import duckdb
from broflow import state
state.set("debug", False)

conn = duckdb.connect()
conn.register("customers_df", customers_df)
conn.register("products_df", products_df)
conn.register("purchases_df", purchases_df)

# One-shot mode
agent = BroInsight(OpenAI20b(), metadata_loader=loader, db=conn)
result = agent.ask("Who are the top 5 most frequent buyers?")


Welcome to BroInsight!
Thank you for using BroInsight!
AI: Here are the five customers with the highest purchase counts (most frequent buyers):

| Rank | Customer ID | Purchase Count |
|------|-------------|----------------|
| 1    | 464         | 15 |
| 2    | 417         | 13 |
| 3    | 637         | 12 |
| 4    | 623         | 12 |
| 5    | 640         | 12 |

*Note:* Customers 637, 623, and 640 all have the same purchase count (12). If you’d like to see additional details (e.g., purchase dates, product categories) or perform further segmentation, just let me know!


In [19]:
# Interactive mode
agent = BroInsight(OpenAI20b(), metadata_loader=loader, db=conn)
result = agent.chat()  # Starts interactive session

Welcome to BroInsight!
AI: **Top 5 most frequent buyers (by purchase count)**

| Rank | Customer ID | Purchase Count | Age | Gender | City        | State | Segment  | Income | Loyalty | Preferred Channel |
|------|-------------|----------------|-----|--------|-------------|-------|----------|--------|---------|------------------|
| 1    | 464         | **15**          | 46  | F      | Chicago     | FL    | Basic    | 95,813 | 52      | Mobile            |
| 2    | 417         | **13**          | 50  | M      | Houston     | CA    | Standard | 73,257 | 17      | Online            |
| 3    | 623         | **12**          | 56  | M      | Los Angeles | NY    | Basic    | 20,876 | 99      | Mobile            |
| 4    | 637         | **12**          | 31  | M      | New York    | NY    | Basic    | 76,138 | 74      | Store             |
| 5    | 640         | **12**          | 64  | F      | Chicago     | CA    | Premium  | 76,762 | 2       | Store             |

*All five customers in the 

Shared(user_input='/exit', metadata_loader=<broinsight.metadata.metadata_loader.MetadataLoader object at 0x12652eae0>, selected_metadata=['purchases_df', 'customers_df'], sql_query='\nWITH purchase_counts AS (\n    SELECT\n        customer_id,\n        COUNT(*) AS purchase_count\n    FROM\n        purchases_df\n    GROUP BY\n        customer_id\n)\nSELECT\n    pc.customer_id,\n    pc.purchase_count,\n    c.age,\n    c.gender,\n    c.city,\n    c.state,\n    c.customer_segment,\n    c.income,\n    c.loyalty_score,\n    c.preferred_channel\nFROM\n    purchase_counts pc\nJOIN\n    customers_df c\n    ON pc.customer_id = c.customer_id\nORDER BY\n    pc.purchase_count DESC\nLIMIT 5;\n', query_result=None, messages=[{'role': 'user', 'content': [{'text': 'Who are the top 5 most frequent buyers?'}]}, {'role': 'assistant', 'content': [{'text': "**Top 5 most frequent buyers (by purchase count)**\n\n| Rank | Customer ID | Purchase Count | Age | Gender | City        | State | Segment  | Income | L